In [0]:
%pip install gitpython -q
dbutils.library.restartPython()

In [0]:
catalog_name = "academy"
schema_name = "labs_lakeflow"
volume = "raw_data"

vol_path = f"/Volumes/{catalog_name}/{schema_name}/{volume}"

In [0]:
repo_url = "https://github.com/Databricks-BR/lab_outubro_2025.git/"

In [0]:
def carga_dos_dados(carga):
    import git
    import os
    import shutil
    import tempfile

    if carga != "initial" and carga != "incremental":
        raise ValueError("Invalid `carga` value. Must be 'initial' or 'incremental'.")

    dbutils.fs.mkdirs(vol_path)

    with tempfile.TemporaryDirectory() as tmp_dir:
        repo = git.Repo.clone_from(repo_url, tmp_dir, depth=1, no_checkout=True)
        repo.git.checkout('HEAD', '--', 'Files/' + carga)
        src = os.path.join(tmp_dir, 'Files/' + carga)
        for item in os.listdir(src):
            item_path = os.path.join(src, item)
            if os.path.isdir(item_path):
                dst = os.path.join(vol_path, item)
                if os.path.exists(dst):
                    shutil.rmtree(dst)
                shutil.copytree(item_path, dst)

    display(dbutils.fs.ls(vol_path))